In [1]:
# Filepaths, numpy, and Tensorflow
import os

import numpy as np
np.random.seed(0)

In [2]:
# Load list of clouds in order
cwd_path = os.getcwd()
images_path = os.path.join(cwd_path, 'image_dataset/dataset')

# List the clouds in the dataset folder: 10 Clouds
list_clouds = [name for name in os.listdir(images_path)]
list_clouds.remove('.DS_Store')
list_clouds.sort()
list_clouds

['altocumulus',
 'altostratus',
 'cirrocumulus',
 'cirrostratus',
 'cirrus',
 'cumulonimbus',
 'cumulus',
 'nimbostratus',
 'stratocumulus',
 'stratus',
 'unknown']

In [3]:
# Load numpy saved XY_train dataset
cwd_path = os.getcwd()
outfile_path = os.path.join(cwd_path, 'XY_train')

# Initialize numpy clouds data with all zeros
X = np.zeros((1,224,224,3))
y = np.zeros((1,1))

# Merge all cloud datasets in X, y
for cloud in list_clouds:
    temp_X = np.load(os.path.join(outfile_path, f"{cloud}_X.npy"))
    temp_y = np.load(os.path.join(outfile_path, f"{cloud}_y.npy"))
    X = np.append(X, temp_X, axis=0)
    y = np.append(y, temp_y, axis=0)

In [4]:
# Peek at X, y shapes
print(X.shape)
print(y.shape)

(3066, 224, 224, 3)
(3066, 1)


In [5]:
# split 80% train and 20% test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

Using TensorFlow backend.
/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
from keras.applications.densenet import DenseNet121
model = DenseNet121(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=11)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [9]:
model.compile(optimizer='adam', 
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [10]:
model.fit(
    X_train,
    y_train_categorical,
    epochs=45,
    shuffle=True,
    verbose=2
)

Epoch 1/45


AbortedError: Operation received an exception:Status: 3, message: could not create a reorder primitive descriptor, in file tensorflow/core/kernels/mkl_concat_op.cc:813
	 [[Node: conv2_block1_concat/concat = _MklConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _kernel="MklOp", _device="/job:localhost/replica:0/task:0/device:CPU:0"](pool1/MaxPool, conv2_block1_2_conv/convolution, training/Adam/gradients/conv2_block4_concat/concat_grad/mod, pool1/MaxPool:2, conv2_block1_2_conv/convolution:2, DMT/_50)]]